In [1]:
#!pip install keras

In [2]:
from module_2_preprocessing import Data_Preprocessing
from module_12_DF_creation import DataFrame_Creation

In [3]:
import warnings
warnings.filterwarnings("ignore")
import joblib
import pandas as pd
import numpy as np
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras import utils as np_utils
from tensorflow.keras.layers import Dropout

In [4]:
class Abstract_based_sentiment_analysis:
    def __init__(self) : 
        self.path_acbsa = "saved_model/acbsa_model.h5"
        self.path_sentiment = "saved_model/sentiment_model.h5"
        self.path_tokenizer = 'saved_model/tokenizer'
        self.path_le_acbsa = 'saved_model/label_encoder_acbsa'
        self.path_le_sentiment = 'saved_model/label_encoder_sentiment'
        self.acbsa_model = self.acbsa_model_creation()
        self.sentiment_model = self.sentiment_model_creation()
        self.dp = Data_Preprocessing()
        self.dfc = DataFrame_Creation()
        
        with open(self.path_tokenizer, 'rb') as f:               
            self.tokenizer = pickle.load(f)
        
        with open(self.path_le_acbsa, 'rb') as f:               
            self.label_encoder_acbsa = pickle.load(f)
        
        with open(self.path_le_sentiment, 'rb') as f:               
            self.label_encoder_sentiment = pickle.load(f)
    
    def acbsa_model_creation(self):
        acbsa_model = Sequential()                                                   
        acbsa_model.add(Dense(512, input_shape=(6000,), activation='relu'))
        acbsa_model.add((Dense(256, activation='relu')))
        acbsa_model.add((Dropout(0.3)))
        acbsa_model.add((Dense(128, activation='relu')))
        acbsa_model.add(Dense(5, activation='softmax'))
        acbsa_model.load_weights(self.path_acbsa)
        return acbsa_model
    
    def sentiment_model_creation(self):
        sentiment_model = Sequential()
        sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
        sentiment_model.add((Dense(256, activation='relu')))
        sentiment_model.add((Dropout(0.3)))
        sentiment_model.add((Dense(128, activation='relu')))
        sentiment_model.add(Dense(4, activation='softmax'))
        sentiment_model.load_weights(self.path_sentiment)
        return sentiment_model 
    
    def predict(self,sentence) :
        sentence_preprocessed = self.dp.preprocess_text(sentence)
        sen_tokenized = pd.DataFrame(self.tokenizer.texts_to_matrix(sentence_preprocessed))
        predicted_cat = self.label_encoder_acbsa.inverse_transform(self.acbsa_model.predict_classes(sen_tokenized))        
        predicted_polarity = self.label_encoder_sentiment.inverse_transform(self.sentiment_model.predict_classes(sen_tokenized))
        result = self.dfc.create_result_dataframe(predicted_cat,predicted_polarity)
        return result


In [5]:
restuarnt_data = pd.read_csv("restaurant_train_data_semeval_14.csv")
reviews = restuarnt_data["text"]
reviews = reviews[0:100]
#reviews

In [6]:
ab = Abstract_based_sentiment_analysis()
result = ab.predict(reviews)
result

100%|██████████| 100/100 [00:02<00:00, 47.18it/s]


predicted_polarity,Ranking,negative %,positive %,neutral %,conflict %
predicted_catagories,,,,,
ambience,0.65,15.384615,76.923077,7.692308,0.000000
anecdotes/miscellaneous,1.70,20.588235,58.823529,17.647059,2.941176
food,1.60,12.500000,65.625000,12.500000,9.375000
price,0.45,22.222222,77.777778,0.000000,0.000000
service,0.60,25.000000,58.333333,0.000000,16.666667
